In [1]:
# import sys
# sys.path.insert(0, '..')
from semantic_mpc_interface import (
    LoadModel,
    get_thermostat_data,
    HPFlexSurvey,
    convert_units,
    SHACLHandler,
    # add_connection
)
from buildingmotif.namespaces import BRICK, RDF, S223
from rdflib import URIRef
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library
import csv
from pyshacl.rdfutil import clone

# SELECT ONTOLOGY 
ontology = 's223'

CRITICAL:root:Install the 'bacnet-ingress' module, e.g. 'pip install buildingmotif[bacnet-ingress]'


# Testing the Model Builder

ModelBuilder has been deprecated in favor of just survey based workflow. May be readded in the future

# Testing the Survey Workflow

In [2]:
# Please disregard excessive outputs (logging and warnings)
# TODO: Figure out how to configure these in building motif 
import logging
logging.disable(logging.CRITICAL)
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Creating survey, allow overwrite if there is something already there
s = HPFlexSurvey(f'{ontology}-test_site','test_build','.', overwrite=True, ontology=ontology,
                 template_dict = {'zone': 'hvac-zone',
                                  "space": "space",
                                  "hvac": "hp-rtu",
                                  "tstat": "tstat",
                                  "window": "window",
                                  "site": "site" })

# Generating a simple building structure that prefills csv files. 
s.easy_config(zone_space_window_list=[(2,2),(1,2),(1,3)])
# Now check test_site/test_build to look at survey files


removing dependency:  space
removing dependency:  window
removing dependency:  hp-rtu
removing dependency:  tstat
values: dict_values([])
values: dict_values(['area-value'])
values: dict_values(['cooling_COP-value', 'cooling_capacity-value', 'heating_COP-value', 'heating_capacity-value'])
values: dict_values(['active-value', 'setpoint_deadband-value', 'resolution-value', 'stage_count-value', 'tolerance-value'])
values: dict_values(['tilt-value', 'area-value', 'azimuth-value'])
values: dict_values(['noaastation-value', 'timezone-value', 'longitude-value', 'latitude-value'])
{'site_id': 's223-test_site', 'hvac_type': 'hp-rtu', 'variable_params': {'hvac-zone': {}, 'space': {'area': '<name>-area'}, 'hp-rtu': {'cooling_capacity': '<name>-cooling_capacity', 'heating_COP': '<name>-heating_COP', 'cooling_COP': '<name>-cooling_COP', 'heating_capacity': '<name>-heating_capacity'}, 'tstat': {'tolerance': '<name>-tolerance', 'active': '<name>-active', 'resolution': '<name>-resolution', 'stage_coun

In [4]:
# Will just fill the columns programmatically for testing, csv should be filled out otherwise
import sys
sys.path.insert(0,'../examples')
from example_prefill_usage import prefill_csv_survey

In [5]:
prefill_csv_survey(f'{ontology}-test_site/test_build')

Loading config from: s223-test_site/test_build/config.json
Found CSV files: ['zone', 'site', 'window', 'hvac', 'point_list', 'tstat', 'space']
Filled 10 empty cells in zone.csv
Filled 4 empty cells across 4 columns in site.csv
Filled 21 empty cells across 3 columns in window.csv
Filled 12 empty cells across 4 columns in hvac.csv
No empty cells found in point_list.csv
Filled 15 empty cells across 5 columns in tstat.csv
Filled 4 empty cells across 1 columns in space.csv


In [6]:
# Reading csv
s.read_csv()

Expanded CSV shape: (6, 5)
New columns added: []
Expanded CSV shape: (4, 4)
New columns added: ['area']
Expanded CSV shape: (3, 13)
New columns added: ['cooling_capacity', 'heating_COP', 'cooling_COP', 'heating_capacity']
Expanded CSV shape: (3, 14)
New columns added: ['tolerance', 'active', 'resolution', 'stage_count', 'setpoint_deadband']
Expanded CSV shape: (7, 10)
New columns added: ['azimuth', 'tilt', 'area']
Expanded CSV shape: (1, 9)
New columns added: ['noaastation', 'longitude', 'latitude', 'timezone']


# Testing SHACL Generation and Validation

In [7]:
og = clone.clone_graph(s.graph)
# Create handler
handler = SHACLHandler(ontology=ontology)

# Generate shapes
handler.generate_shapes()

# Save shapes
handler.save_shapes('shapes.ttl')

# Run inference on model
inferred_graph = handler.infer(s.graph)

inferred_graph.serialize(f'{ontology}-test_site/test_build/reasoned.ttl', format = 'ttl')
# if not validation_result.valid:
#     print("Validation failed:")
#     print(validation_result.report_string)

<Graph identifier=b0a238d1-9e67-4183-879d-04ae8ae1f1bb (<class 'rdflib.graph.Graph'>)>

In [8]:
# lots of new inferred information
(inferred_graph-og).print()

@prefix ns1: <urn:hpflex/shapes#> .

<urn:hpflex/s223-test_site#hvac_1> a ns1:hp-rtu ;
    ns1:air-connected-to <urn:hpflex/s223-test_site#zone_1> ;
    ns1:has-point <urn:hpflex/s223-test_site#hvac_1-cooling_COP>,
        <urn:hpflex/s223-test_site#hvac_1-cooling_capacity>,
        <urn:hpflex/s223-test_site#hvac_1-heating_COP>,
        <urn:hpflex/s223-test_site#hvac_1-heating_capacity> .

<urn:hpflex/s223-test_site#hvac_2> a ns1:hp-rtu ;
    ns1:air-connected-to <urn:hpflex/s223-test_site#zone_2> ;
    ns1:has-point <urn:hpflex/s223-test_site#hvac_2-cooling_COP>,
        <urn:hpflex/s223-test_site#hvac_2-cooling_capacity>,
        <urn:hpflex/s223-test_site#hvac_2-heating_COP>,
        <urn:hpflex/s223-test_site#hvac_2-heating_capacity> .

<urn:hpflex/s223-test_site#hvac_3> a ns1:hp-rtu ;
    ns1:air-connected-to <urn:hpflex/s223-test_site#zone_3> ;
    ns1:has-point <urn:hpflex/s223-test_site#hvac_3-cooling_COP>,
        <urn:hpflex/s223-test_site#hvac_3-cooling_capacity>,
        

# Testing get Metadata

In [9]:
# still working on loader, will clean up class, but functionality about right
loader = LoadModel(f"{ontology}-test_site/test_build/reasoned.ttl", 
                   template_dict={'sites':'site', 'zones': 'hvac-zone'}, 
                   ontology = ontology)
site_info = loader.get_all_building_objects()

latitude
longitude
http://data.ashrae.org/standard223#hasValue 1.0
http://www.w3.org/1999/02/22-rdf-syntax-ns#type urn:hpflex/shapes#timezone
timezone
http://www.w3.org/1999/02/22-rdf-syntax-ns#type urn:hpflex#Timezone
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://data.ashrae.org/standard223#EnumeratedProperty
http://data.ashrae.org/standard223#hasValue 1.0
http://www.w3.org/1999/02/22-rdf-syntax-ns#type urn:hpflex/shapes#noaastation
noaastation
http://www.w3.org/1999/02/22-rdf-syntax-ns#type urn:hpflex#NOAAStation
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://data.ashrae.org/standard223#EnumeratedProperty
http://qudt.org/schema/qudt/hasUnit http://qudt.org/vocab/unit/Degree
http://qudt.org/schema/qudt/hasQuantityKind http://qudt.org/vocab/quantitykind/GeneralizedCoordinate
http://data.ashrae.org/standard223#hasValue 1.0
http://www.w3.org/1999/02/22-rdf-syntax-ns#type urn:hpflex/shapes#longitude
longitude
http://www.w3.org/1999/02/22-rdf-syntax-ns#type urn:hpflex#Long

In [10]:
print(site_info)

{}


In [11]:
zone = site_info['zones'][0]
print(zone)

KeyError: 'zones'

In [ ]:
zone.spaces

[Space(name='urn:hpflex/s223-test_site#space_3_1', area=Value(value=4.0, unit='http://qudt.org/vocab/unit/FT2'), space=urn:hpflex/s223-test_site#space_3_1)]

In [ ]:
zone.windows[0].area.name

rdflib.term.URIRef('urn:hpflex/s223-test_site#window_3_3-area')

In [ ]:
zone.tstats[0]

Tstat(name='urn:hpflex/s223-test_site#tstat_zone_3', resolution=Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F'), active=Value(value=3.0, unit='None'), setpoint_deadband=Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F'), tolerance=Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F'), stage_count=Value(value=3.0, unit='None'), tstat=urn:hpflex/s223-test_site#tstat_zone_3)

In [ ]:
print(zone.tstats[0].resolution)
zone.tstats[0].resolution.convert_to_si()
print(zone.tstats[0].resolution)
print(zone.tstats[0].resolution.is_delta)

Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F')
Value(value=1.6666666666666667, unit='http://qudt.org/vocab/unit/DEG_C')
True


In [ ]:
# optionally just load everything as si 
si_loader = LoadModel(f"{ontology}-test_site/test_build/test_build.ttl", ontology = 'brick', as_si_units=True)
site_info = si_loader.get_all_building_objects()
print(zone.tstats[0].resolution)

Value(value=1.6666666666666667, unit='http://qudt.org/vocab/unit/DEG_C')


In [ ]:
# Getting the thermostat metadata
get_thermostat_data(si_loader)

{'heat_availability': [],
 'cool_availability': [],
 'heat_tolerance': [],
 'cool_tolerance': [],
 'setpoint_deadband': [],
 'active': [],
 'control_group': [],
 'control_type_list': [],
 'floor_area_list': [],
 'floor_area_unit': [],
 'window_area_list': [],
 'window_area_unit': [],
 'azimuth_list': [],
 'azimuth_unit': [],
 'tilt_list': [],
 'tilt_unit': [],
 'zone_ids': [],
 'hvacs': [],
 'setpoint_type': [],
 'fuel_heat_list': [],
 'fuel_cool_list': [],
 'cooling_capacity': [],
 'cooling_capacity_unit': [],
 'heating_capacity': [],
 'heating_capacity_unit': [],
 'cooling_cop': [],
 'heating_cop': [],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': [],
 'temperature_unit': []}

In [ ]:
# Getting the thermostat metadata for 1 zone 
get_thermostat_data(si_loader, ['zone_1','zone_2'])

{'heat_availability': [],
 'cool_availability': [],
 'heat_tolerance': [],
 'cool_tolerance': [],
 'setpoint_deadband': [],
 'active': [],
 'control_group': [],
 'control_type_list': [],
 'floor_area_list': [],
 'floor_area_unit': [],
 'window_area_list': [],
 'window_area_unit': [],
 'azimuth_list': [],
 'azimuth_unit': [],
 'tilt_list': [],
 'tilt_unit': [],
 'zone_ids': [],
 'hvacs': [],
 'setpoint_type': [],
 'fuel_heat_list': [],
 'fuel_cool_list': [],
 'cooling_capacity': [],
 'cooling_capacity_unit': [],
 'heating_capacity': [],
 'heating_capacity_unit': [],
 'cooling_cop': [],
 'heating_cop': [],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': [],
 'temperature_unit': []}

# Testing Unit Conversion

In [ ]:
convert_units(10, 'FT', 'M')

3.048

In [ ]:
convert_units(0, 'DEG_C', 'DEG_F')

31.999999999999886

In [ ]:
convert_units(0, 'DEG_C', 'K')

273.15

In [ ]:
# Planning to implement converting everything to si in loader
# loader.get_all_building_objects

# Testing Grafana Dashboarding 
Needs updates

In [ ]:
# import yaml 
# with open('../development_files/creds.yml') as f:
#     config = yaml.safe_load(f)

# bg = BrickToGrafana(grafana_server=config['grafana_server'], grafana_api_key = config['grafana_api_key'], datasource=config['datasource'], ttl_path = 'test-brick-model.ttl')

In [ ]:
# bg.create_dashboard('AQL from brick')

In [ ]:
# Grafana not set up, upload won't work
# bg.upload_dashboard(message = 'testing upload')